# DreamJob Reviews Parser

In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time


def extract_review_data(driver, review):
    review_id = review.get_attribute('id')
    header = review.find_element(By.CLASS_NAME, 'review__header').text.strip()
    date = review.find_element(By.CLASS_NAME, 'review__date').text.strip()

    # Extract HTML content of the review
    review_html_content = review.get_attribute('outerHTML')

    # Use BeautifulSoup to parse the HTML content
    soup = BeautifulSoup(review_html_content, 'html.parser')

    # Extracting the first three tags (skip them)
    tags_header = soup.find("div", class_="tags__header")
    if tags_header:
        return None  # Skip the review if the header is present

    # Extract header tags
    header_tags = soup.find("div", class_="tags")
    header_tags_list = [tag.text.strip() for tag in header_tags.find_all("div", class_="tags__item")]

    # Extract perks tags
    perks_tags_span = soup.find("span", class_="review__perk-title", data_partly_switch="")
    if perks_tags_span:
        perks_tags_div = perks_tags_span.find_next("div", class_="tags")
        perks_tags_list = [perk.text.strip() for perk in perks_tags_div.find_all("div", class_="tags__item")]
    else:
        perks_tags_list = []

    rating_div = review.find_element(By.CLASS_NAME, 'dj-rating')
    rating_data = rating_div.get_attribute('data-partly-switch')

    review_text_elements = review.find_elements(By.CLASS_NAME, 'review__text')
    review_texts = [text_element.text.strip() for text_element in review_text_elements]

    # Combine text blocks into a single string
    separator = '<separator>'  # Используйте ваш собственный разделитель
    review_text = separator.join(review_texts)

    helpful_count = review.find_element(By.CLASS_NAME, 'bt--32').find_element(By.TAG_NAME, 'span').text.strip()

    # Initialize employer response variables
    employer_response_date = None
    employer_response_title = None
    employer_response_msg = None

    # Attempt to extract employer response
    try:
        response_element = review.find_element(By.CLASS_NAME, 'review__answer')
        employer_response_date = response_element.find_element(By.CLASS_NAME, 'review__answer-date').text.strip()
        employer_response_title = response_element.find_element(By.CLASS_NAME, 'review__answer-title').text.strip()
        employer_response_msg = response_element.find_element(By.CLASS_NAME, 'review__answer-msg').text.strip()
    except NoSuchElementException:
        pass  # No employer response found

    # Extract review link
    review_link = review.find_element(By.XPATH, './/a[@data-clipboard]').get_attribute('data-clipboard')

    return {
        'Review_ID': review_id,
        'Header': header,
        'Date': date,
        'Header_Tags': header_tags_list,
        'Perks_Tags': perks_tags_list,
        'Rating_Data': rating_data,
        'Review_Text': review_text,
        'Helpful_Count': helpful_count,
        'Employer_Response_Date': employer_response_date,
        'Employer_Response_Title': employer_response_title,
        'Employer_Response_Msg': employer_response_msg,
        'Review_Link': review_link
    }

def scroll_to_reviews_end(driver):
    # Находим последний отзыв
    last_review = driver.find_elements(By.CLASS_NAME, 'review')[-1]
    
    # Прокручиваем страницу до последнего отзыва
    driver.execute_script("arguments[0].scrollIntoView();", last_review)
    
    driver.execute_script("window.scrollBy(0, 350);") # Ещё немного докручиваем
    
    # Ждем некоторое время, чтобы дать новым отзывам загрузиться
    time.sleep(0.2) # Можно уменьшить или убрать
    
def get_source_html(url, output_file='output_reviews_Vse_instrumenty.csv'): ## ИМЯ ВЫХОДНОГО ФАЙЛА
    global driver
    service = ChromeService('/chromedriver-mac-arm64_33/chromedriver') # ПУТЬ К CHROMEDRIVER
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=service, options=options)

    try:
        driver.maximize_window()
        driver.get(url)
        time.sleep(3)

        while True:
            scroll_to_reviews_end(driver)
            time.sleep(0.2)

            loader_elements = driver.find_elements(By.CLASS_NAME, 'ajax-loader')

            if any("display: none;" in element.get_attribute("style") for element in loader_elements):
                print("All reviews are loaded, exiting the loop")
                break

        # Extract reviews and create a DataFrame
        reviews = driver.find_elements(By.CLASS_NAME, 'review')
        data = []
        for review in reviews:
            try:
                review_data = extract_review_data(driver, review)
                data.append(review_data)
                print("Review data extracted successfully")
            except Exception as e:
                print(f"Error extracting review data: {e}")

        df = pd.DataFrame(data)
        df.to_csv(output_file, index=False)
        print(f"DataFrame saved to {output_file}")

    except Exception as e:
        print(f"An error occurred: {e}")

    finally:
        driver.close()
        driver.quit()

def main():
    url = 'https://dreamjob.ru/employers/30831' # ССЫЛКА НА СТРАНИЦУ КОМПАНИИ
    get_source_html(url)

if __name__ == "__main__":
    main()

All reviews are loaded, exiting the loop
Review data extracted successfully
Review data extracted successfully
Review data extracted successfully
Review data extracted successfully
Review data extracted successfully
Review data extracted successfully
Review data extracted successfully
Review data extracted successfully
Review data extracted successfully
Review data extracted successfully
Review data extracted successfully
Review data extracted successfully
Review data extracted successfully
Review data extracted successfully
Review data extracted successfully
Review data extracted successfully
Review data extracted successfully
Review data extracted successfully
Review data extracted successfully
Review data extracted successfully
Review data extracted successfully
Review data extracted successfully
Review data extracted successfully
Review data extracted successfully
Review data extracted successfully
Review data extracted successfully
Review data extracted successfully
Review data ex

## Обрабаботаем данные для удобства анализа

In [26]:
import numpy as np
import pandas as np

df = pd.read_csv('output_reviews_Vse_instrumenty.csv')
df.head()

,Review_ID,Header,Date,Header_Tags,Perks_Tags,Rating_Data,Review_Text,Helpful_Count,Employer_Response_Date,Employer_Response_Title,Employer_Response_Msg,Review_Link
0,review2031687,Специалист по логистике,Октябрь 2023,"['Бывший сотрудник', 'Стаж в компании: 3-5 лет...","['Своевременная оплата труда', 'Удобное распол...","rating|4,5,5,5,3,5","Отличное руководство, карьерный рост, чистое, ...",2,Октябрь 2023,Ответ представителя компании,Добрый день!\nСпасибо за оставленный отзыв. Мы...,https://dreamjob.ru/employers/30831?review_id=...
1,review2193814,Аналитик,Декабрь 2023,"['Работаю в компании', 'Стаж в компании: Меньш...","['Своевременная оплата труда', 'Удобное распол...","rating|5,5,5,5,5,5",Отличное место работы! Атмосфера и взаимоотнош...,1,Декабрь 2023,Ответ представителя компании,Добрый день!\nСпасибо за оставленный отзыв. Мы...,https://dreamjob.ru/employers/30831?review_id=...
2,review2193923,Оператор,Декабрь 2023,"['Работаю в компании', 'Стаж в компании: 1-2 г...",[],"rating|5,4,5,4,5,5","Трансфер, своевременная оплата труда, ДМС, спо...",0,Декабрь 2023,Ответ представителя компании,Добрый день!\nСпасибо за оставленный отзыв. Ск...,https://dreamjob.ru/employers/30831?review_id=...
3,review2216697,Руководитель отдела,Декабрь 2023,"['Работаю в компании', 'Стаж в компании: 1-2 г...","['Своевременная оплата труда', 'Удобное распол...","rating|5,5,5,5,5,5","В компании много возможностей для роста, регул...",0,Январь 2024,Ответ представителя компании,Добрый день!\nСпасибо за оставленный отзыв. Мы...,https://dreamjob.ru/employers/30831?review_id=...
4,review2424155,Менеджер по продажам,Март 2024,"['Бывший сотрудник', 'Стаж в компании: Меньше ...","['Своевременная оплата труда', 'Удобное распол...","rating|5,5,5,4,5,4","Достойный уровень зп, близость к дому, мотивац...",0,NaN,NaN,NaN,https://dreamjob.ru/employers/30831?review_id=...


In [27]:
df['Rating_Data'] = df['Rating_Data'].str.replace('rating\|', '', regex=True)

def create_rating_columns(row):
    # Split the string into a list of integers using commas as the delimiter
    ratings = list(map(int, row.split(',')))

    # Create a pandas Series with specific column names for each rating category
    return pd.Series({
        'Working Conditions': ratings[0],
        'Income Level': ratings[1],
        'Team': ratings[2],
        'Management': ratings[3],
        'Recreation Opportunities': ratings[4],
        'Growth Opportunities': ratings[5]
    }) 

# Применяем функцию и создаем новые колонки
df = df.join(df['Rating_Data'].apply(create_rating_columns))

In [28]:
# Let's create a dictionary for converting textual months into numerical format
month_dict = {
'January': '01', 'February': '02', 'March': '03',
'April': '04', 'May': '05', 'June': '06',
'July': '07', 'August': '08', 'September': '09',
'October': '10', 'November': '11', 'December': '12'
}
# Convert the 'Date' column
df['Date'] = df['Date'].apply(lambda x: f"{month_dict[x.split()[0]]}.{x.split()[1]}")


KeyError: 'Октябрь'

In [29]:
import numpy as np

In [30]:
# Создадим словарь для преобразования текстовых месяцев в числовой формат
month_dict = {
    'Январь': '01', 'Февраль': '02', 'Март': '03',
    'Апрель': '04', 'Май': '05', 'Июнь': '06',
    'Июль': '07', 'Август': '08', 'Сентябрь': '09',
    'Октябрь': '10', 'Ноябрь': '11', 'Декабрь': '12'
}

# Преобразование столбца 'Date'
df['Employer_Response_Date'] = df['Employer_Response_Date'].apply(lambda x: f"{month_dict[x.split()[0]]}.{x.split()[1]}" if pd.notna(x) else x)


In [31]:
df['Review_ID'] = df['Review_ID'].str.replace('review', '', regex=True)

In [32]:
df.head(2)

,Review_ID,Header,Date,Header_Tags,Perks_Tags,Rating_Data,Review_Text,Helpful_Count,Employer_Response_Date,Employer_Response_Title,Employer_Response_Msg,Review_Link,Working Conditions,Income Level,Team,Management,Recreation Opportunities,Growth Opportunities
0,2031687,Специалист по логистике,Октябрь 2023,"['Бывший сотрудник', 'Стаж в компании: 3-5 лет...","['Своевременная оплата труда', 'Удобное распол...","4,5,5,5,3,5","Отличное руководство, карьерный рост, чистое, ...",2,10.2023,Ответ представителя компании,Добрый день!\nСпасибо за оставленный отзыв. Мы...,https://dreamjob.ru/employers/30831?review_id=...,4,5,5,5,3,5
1,2193814,Аналитик,Декабрь 2023,"['Работаю в компании', 'Стаж в компании: Меньш...","['Своевременная оплата труда', 'Удобное распол...","5,5,5,5,5,5",Отличное место работы! Атмосфера и взаимоотнош...,1,12.2023,Ответ представителя компании,Добрый день!\nСпасибо за оставленный отзыв. Мы...,https://dreamjob.ru/employers/30831?review_id=...,5,5,5,5,5,5


In [33]:
df.to_csv('Vse_instrumenty.csv')

In [11]:
df.dtypes

Review_ID                    object
Header                       object
Date                         object
Header_Tags                  object
Perks_Tags                   object
Rating_Data                  object
Review_Text                  object
Helpful_Count                 int64
Employer_Response_Date      float64
Employer_Response_Title     float64
Employer_Response_Msg       float64
Review_Link                  object
Working Conditions            int64
Income Level                  int64
Team                          int64
Management                    int64
Recreation Opportunities      int64
Growth Opportunities          int64
dtype: object